In [ ]:
import pandas as pd
import ipywidgets as widgets
from bs4 import BeautifulSoup
import requests
from IPython.core.display import display, HTML

In [ ]:
display(
    HTML(
        """
<script>
    // The below callback is used to toggle code
    code_show=true; 
    function code_toggle() {
         if (code_show){
         $('div.input').hide();
         } else {
         $('div.input').show();
         }
         code_show = !code_show
    } 
    $( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()">
    <input type="submit" value="Toggle Code">
</form>
<script>
    IPython.OutputArea.auto_scroll_threshold = 9999;
</script>
"""
    )
)
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
data = pd.read_csv("../data/processed/train.csv", sep=",")


def get_user_recoms(price, depreciation, vehicle_type, top_n):
    return data[:top_n]


def get_similar_items(listing_id, top_n=10):
    return data[:top_n]

In [ ]:
SELECT = "Select"
price_ranges = [
    SELECT,
    "Below $10,000",
    "$10,001 to $20,000",
    "$20,001 to $30,000",
    "$30,001 to $40,000",
    "$40,001 to $50,000",
    "$50,001 to $60,000",
    "$60,001 to $70,000",
    "$70,001 to $80,000",
    "$80,001 to $100,000",
    "$100,001 to $120,000",
    "$120,001 to $140,000",
    "$140,001 to $160,000",
    "$160,001 to $180,000",
    "$180,001 to $200,000",
    "Above $200,000",
]
depreciation_ranges = [
    SELECT,
    "Below $10k /yr",
    "$10k to $11k /yr",
    "$11k to $12k /yr",
    "$12k to $13k /yr",
    "$13k to $14k /yr",
    "$14k to $16k /yr",
    "$16k to $18k /yr",
    "$18k to $20k /yr",
    "$20k to $25k /yr",
    "Above $25k /yr",
]
type_of_vehicle_ranges = [
    SELECT,
    "sports car",
    "luxury sedan",
    "suv",
    "hatchback",
    "mid-sized sedan",
    "stationwagon",
    "mpv",
    "bus/mini bus",
    "truck",
    "others",
    "van",
]
top_n_values = [5, 10, 20, 50]

In [ ]:
def get_car_thumbnail_url(listing_id):

    response = requests.get(
        f"https://www.sgcarmart.com/used_cars/info.php?ID={listing_id}"
    )
    try:
        soup = BeautifulSoup(response.text, features="html.parser")
        image_thumbnail = soup.find(id="gallery").contents[0].find("img").attrs["src"]
    except Exception as e:
        image_thumbnail = None

    return image_thumbnail


def construct_user_result_table(df_row):
    img_url = get_car_thumbnail_url(df_row.listing_id)
    return widgets.HTML(
        f"""
    <table>
        <tbody>
            <tr>
                <td></td>
                <td><b>{df_row.title.title()} </b></td>
                <td></td>
                <td></td>
            </tr>
            <tr>
                <td> <img src="{img_url}" alt="{"Car Image" if img_url else "Image Not Available"}" style="height: 150px; width:200px"/> </td>
                <td></td>
                <td><b>Listing ID</b></td>
                <td style="padding-left: 5px">{df_row.listing_id}</td>
            </tr>
            <tr>
                <td><b>Price</b></td>
                <td style="color: red">{df_row.price}</td>
                <td><b>Depreciation</b></td>
                <td style="padding-left: 5px">{df_row.depreciation}</td>
            </tr>
            <tr>
                <td><b>Vehicle Type</b></td>
                <td>{df_row.type_of_vehicle.title()}</td>
                <td><b>Dereg Value</b></td>
                <td style="padding-left: 5px">{df_row.dereg_value}</td>
            </tr>
        </tbody>
    </table>
    """
    )

In [ ]:
price_widget = widgets.Dropdown(
    options=price_ranges, description="Price Range", disabled=False,
)
depreciation_widget = widgets.Dropdown(
    options=depreciation_ranges, description="Depreciation Range", disabled=False,
)
depreciation_widget.style.description_width = "120px"
vehicle_type_widget = widgets.Dropdown(
    options=type_of_vehicle_ranges, description="Vehicle Type", disabled=False,
)
top_n_widget = widgets.Dropdown(
    options=top_n_values, description="Show Top n Results", disabled=False,
)
top_n_widget.style.description_width = "120px"
filter_widgets = widgets.HBox(
    [price_widget, depreciation_widget, vehicle_type_widget, top_n_widget]
)

show_results_btn = widgets.Button(description="Find Cars", button_style="info",)

center_contents = widgets.VBox(
    [filter_widgets, widgets.HTML("<br>"), show_results_btn, widgets.HTML("<hr>")]
)
results_area = widgets.Output()

In [ ]:
def process_on_click(btn):
    price = price_widget.value
    depreciation = depreciation_widget.value
    vehicle_type = vehicle_type_widget.value
    top_n = top_n_widget.value

    results_area.clear_output()

    if (price == SELECT) and (depreciation == SELECT) and (vehicle_type == SELECT):
        with results_area:
            display(
                widgets.Label(
                    "Please select at least one of Price Range, Depreciation Range or Vehicle Type"
                )
            )
        return

    user_result_area = widgets.Output()
    similar_items_area = widgets.Output()
    user_df = get_user_recoms(price, depreciation, vehicle_type, top_n)

    with results_area:
        display(
            widgets.HBox(
                [
                    user_result_area,
                    widgets.HTML(
                        """<div style="border-left:1px solid #eeeeee;height:100%"></div>"""
                    ),
                    similar_items_area,
                ]
            )
        )

    def populate_similar_result_area(btn):
        similar_items_area.clear_output()
        listing_id = int(btn.tooltip)
        with similar_items_area:
            display(
                widgets.HTML(
                    f"<h4>You might also like (since you selected listing - {listing_id})</h4>"
                )
            )

        similar_items_df = get_similar_items(listing_id)

        for _, row in similar_items_df.iterrows():
            tbl = construct_user_result_table(row)
            with similar_items_area:
                display(tbl)

    with user_result_area:
        display(widgets.HTML("<h4>Recommended Cars</h4>"))

    for _, row in user_df.iterrows():
        tbl = construct_user_result_table(row)
        show_similar_btn = widgets.Button(
            description="Show Similar", button_style="info", tooltip=str(row.listing_id)
        )
        show_similar_btn.on_click(populate_similar_result_area)
        with user_result_area:
            display(widgets.VBox([tbl, show_similar_btn]))

    with similar_items_area:
        display(
            widgets.HTML("<h6>Select one of Recommended Cars to see similar cars</h6>")
        )


show_results_btn.on_click(process_on_click)

In [ ]:
widgets.VBox(
    [
        widgets.HTML(
            "<h1> Find your next car</h1><h4>Powered by Team Knowledge++</h4>"
        ),
        center_contents,
        results_area,
    ]
)